# Create your own database and fill it with data

#### Create Django project
#### Create first App (place to store your models)

In [2]:
!django-admin startproject myproj1
!cd myproj1 && django-admin startapp myapp1

##### Copy the code below into notebooks/myproj1/myapp1/models.py

In [2]:
from django.db import models

class Profile(models.Model):
    ''' Description of T/S Profile'''
    lon = models.FloatField()
    lat = models.FloatField()

    
class Variable(models.Model):
    ''' Description of Temperature, Salinity, etc '''
    name = models.CharField(max_length=200)
    units = models.CharField(max_length=200)

    
class Measurement(models.Model):
    ''' Description of a measurement '''
    variable = models.ForeignKey(Variable, on_delete=models.CASCADE)
    profile = models.ForeignKey(Profile, on_delete=models.CASCADE)
    depth = models.FloatField()
    value = models.FloatField()

### Edit `notebooks/myproj1/myproj1/settings.py`

In [ ]:
INSTALLED_APPS = (
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'myapp1',                        ### ADD THIS LINE. RESPECT SYNTAX!
)

## Migrate your models to your database

In [6]:
!cd myproj1 && ./manage.py makemigrations && ./manage.py migrate

Migrations for 'myapp1':
  myapp1/migrations/0001_initial.py
    - Create model Profile
    - Create model Variable
    - Create model Measurement
Operations to perform:
  Apply all migrations: admin, auth, contenttypes, myapp1, sessions
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying auth.0001_initial... OK
  Applying admin.0001_initial... OK
  Applying admin.0002_logentry_remove_auto_add... OK
  Applying admin.0003_logentry_add_action_flag_choices... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying auth.0007_alter_validators_add_error_messages... OK
  Applying auth.0008_alter_user_username_max_length... OK
  Applying auth.0009_alter_user_last_name_max_length... OK


## Use the database in IPython Notebook
### Create a Notebook INSIDE `notebooks/myproj1`
#### Copy the code from the cells below into your new notebook 

In [ ]:
import os, sys
print(sys.path.append('/src/notebooks/myproj1/'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "myproj1.settings")
import django
django.setup()

from myapp1.models import Profile, Variable, Measurement

%whos

### Use your models. Create a profile, a variable and a measurement

In [3]:
# create profile and add coordinates
p = Profile()
p.lon = 10
p.lat = 20
p.save()  # save into database

# variable and add name, units
v = Variable()
v.name='salinity'
v.units='psu'
v.save()


# create a measurement
m = Measurement()
m.variable = v
m.profile = p
m.depth = 10
m.value = 35
m.save()


# Use your imagination
# Create many more measurements

# get all measurements
ms = Measurement.objects.all()
print (ms)
print (ms[0].value)
print (ms[0].profile.lon)
print (ms[0].variable.name)

# find only salinities
salOnly = Measurement.objects.filter(variable__name='salinity')

# find only measurements deeper than 5 m
deepMes =  Measurement.objects.filter(depth__gte=5)

[<Measurement: Measurement object>, <Measurement: Measurement object>]
35.0
10.0
salinity


## Run Django web-server (for advanced students)
### Create superuser 
#### In the virtual machine, in `mynotebooks/myptoj1` run command
```
python manage.py createsuperuser
```
#### Enter your desired username and press enter.
```
Username: admin
```
#### Don't add email
```
Email address:
```
#### Add password 'vagrant'
```
Password: **********
Password (again): *********
Superuser created successfully.
```
### Edit `mynotebooks/myproj1/myapp1/admin.py` :

In [ ]:
from django.contrib import admin

# Register your models here.
from .models import Profile, Variable, Measurement

admin.site.register(Profile)
admin.site.register(Variable)
admin.site.register(Measurement)

### Run the server
#### In the virtual machine, in `mynotebooks/myptoj1` run command
```
python manage.py runserver 0.0.0.0:8000
```
#### Open http://192.168.11.33:8000/admin in your browser
#### Enter vagrant/vagrant as username and password
#### Inspect your objects